# APIM ❤️ MCP — Modernize Your APIs for the AI Agent Era

## Expose a REST API as an MCP Server through Azure API Management

> *"Your APIs are becoming tools. Your users are becoming agents. Your platform needs to adapt."*
> — [Azure API Management now supports MCP](https://techcommunity.microsoft.com/blog/integrationsonazureblog/%F0%9F%9A%80-new-in-azure-api-management-mcp-in-v2-skus--external-mcp-compliant-server-sup/4440294)

The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/) is an open standard that turns your APIs into **tools** that AI agents can discover and invoke. This lab demonstrates how to **modernize an existing REST API** — a Product Catalog service — by wrapping it as an MCP server and fronting it with **Azure API Management** as a secure, enterprise-grade control plane.

No code rewrites to your business logic. No new infrastructure paradigms. Just your existing API, now accessible to every MCP-compatible AI agent — with security, monitoring, and governance built in.

### Architecture

```
┌─────────────┐  ┌─────────────┐  ┌─────────────┐
│   Agent      │  │  VS Code    │  │    MCP      │
│  Framework   │  │  Copilot    │  │  Inspector  │
└──────┬───────┘  └──────┬──────┘  └──────┬──────┘
       │                 │                │
       └─────────────────┼────────────────┘
                         │
                         ▼
              ┌─────────────────────┐
              │   Azure API Mgmt    │
              │     (BasicV2)       │
              └───┬─────────────┬───┘
                  │             │
                  ▼             ▼
       ┌──────────────┐  ┌──────────────┐
       │ Container App│  │  AI Foundry  │
       │ (MCP Server) │  │ (GPT-4o-mini)│
       └──────────────┘  └──────────────┘
              ▲
              │ image pull
       ┌──────────────┐
       │     ACR      │
       └──────────────┘
```

📐 Full deep dive → **[ARCHITECTURE.md](ARCHITECTURE.md)**

### What Gets Deployed

| Resource | Purpose |
|---|---|
| **Azure API Management (BasicV2)** | Unified gateway with native MCP support |
| **Azure Container App** | Product Catalog MCP server (FastMCP + Starlette) |
| **Azure Container Registry** | Builds and stores the container image |
| **Azure AI Foundry** | GPT-4o-mini for the Agent Framework demo |
| **Log Analytics + App Insights** | Observability for all resources |

### MCP Tools Exposed

| Tool | Description |
|---|---|
| `get_categories` | List all product categories |
| `get_products_by_category` | Get products in a given category |
| `get_product` | Get product details by ID |
| `search_products` | Search products by name or description |
| `check_stock` | Check stock availability by product ID |

### Lab Steps

| # | Step | Description |
|---|---|---|
| 0️⃣ | [Initialize](#0) | Set variables and generate unique resource names |
| 1️⃣ | [Verify Azure CLI](#1) | Confirm login and subscription |
| 🧪 | [Local test](#localtest) | Validate MCP server locally before deploying |
| 2️⃣ | [Deploy to Azure](#2) | Provision all resources via Bicep |
| 3️⃣ | [Get outputs](#3) | Retrieve deployment URLs and keys |
| 4️⃣ | [Build & deploy](#4) | Build container image and deploy |
| 🧪 | [Test tools via APIM](#testconnection) | Discover and invoke MCP tools through the gateway |
| 🧪 | [Call MCP tools](#mcpcall) | End-to-end tool invocation through APIM |
| 🧪 | [MCP Inspector](#inspector) | Interactive testing in the browser |
| 🧪 | [Agent Framework](#agentframework) | AI agent using MCP tools through APIM |
| 🧪 | [VS Code integration](#vscode) | Configure for GitHub Copilot Agent Mode |
| 🗑️ | [Clean up](#clean) | Delete all Azure resources |

### Prerequisites

- [Python 3.12+](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) with the [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor + RBAC Administrator roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and signed in

### References

- 📝 [Blog: MCP in v2 SKUs + external MCP server support](https://techcommunity.microsoft.com/blog/integrationsonazureblog/%F0%9F%9A%80-new-in-azure-api-management-mcp-in-v2-skus--external-mcp-compliant-server-sup/4440294)
- 📖 [Expose APIs as MCP servers](https://learn.microsoft.com/azure/api-management/expose-mcp-server) · [Connect external MCP servers](https://learn.microsoft.com/azure/api-management/connect-mcp-server) · [Secure MCP access](https://learn.microsoft.com/azure/api-management/secure-mcp-server) · [Discover tools in API Center](https://learn.microsoft.com/azure/api-center/discover-mcp-tools)

▶️ Click **Run All** to execute all steps sequentially, or step through them one at a time.

<a id='0'></a>
### 0️⃣ Initialize notebook variables

Configure the deployment parameters for your environment. Each run generates a **unique suffix** to avoid naming collisions — making it safe to run multiple experiments in parallel.

- Resources are suffixed by a unique string based on your subscription ID
- Adjust `resource_group_location` based on [product availability by Azure region](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management)
- Adjust the OpenAI model and version based on [model availability by region](https://learn.microsoft.com/azure/ai-services/openai/concepts/models)

In [ ]:
import os, sys, json, random, string
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path

# Ensure az CLI is on PATH (pip-installed az may live outside the venv)
az_bin_dir = os.path.expanduser("~/Library/Python/3.12/bin")
if az_bin_dir not in os.environ.get("PATH", ""):
    os.environ["PATH"] = az_bin_dir + os.pathsep + os.environ.get("PATH", "")

import utils

# Generate a unique suffix for this notebook run so every execution creates fresh resources
run_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=6))

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}-{run_suffix}" # unique per run
resource_group_location = "eastus2"

aiservices_config = [{"name": "foundry1", "location": "eastus2"}]

models_config = [{"name": "gpt-4o-mini", "publisher": "OpenAI", "version": "2024-07-18", "sku": "GlobalStandard", "capacity": 100}]

apim_sku = 'Basicv2'
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"},
                             {"name": "subscription2", "displayName": "Subscription 2"}]

inference_api_path = ""  # path to the inference API in the APIM service – use blank for default
inference_api_type = "AzureOpenAI"  # options: AzureOpenAI, AzureAI, OpenAI, PassThrough
inference_api_version = "2025-03-01-preview"
foundry_project_name = deployment_name

build = 1
product_catalog_mcp_server_image = "product-catalog-mcp-server"
product_catalog_mcp_server_src = "../../shared/mcp-servers/product-catalog/http"

utils.print_ok(f'Notebook initialized (run suffix: {run_suffix})')
print(f"👉🏽 Resource Group: {resource_group_name}")

In [ ]:
# Ensure you have the required packages
%pip install mcp openai requests nest_asyncio azure-identity azure-ai-projects

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and connected subscription

Ensure you have the latest Azure CLI and are signed in to the correct subscription. The deployment requires **Contributor** and **RBAC Administrator** roles.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

<a id='localtest'></a>
### 🧪 Local test — Validate the MCP server before deploying

Before deploying to Azure, let's verify the Product Catalog MCP server starts correctly and all 5 tools are discoverable. This runs the same FastMCP + Starlette server that will be containerized and deployed to Azure Container Apps.

The server exposes a **Streamable HTTP** endpoint — the same transport that APIM v2 natively supports.

In [ ]:
import asyncio, threading, time, sys, importlib
sys.path.insert(0, '../../shared/mcp-servers/product-catalog/http')

# Force-reload the module so a fresh ASGI app / session manager is created on re-runs
import mcp_server
importlib.reload(mcp_server)
from mcp_server import app
import uvicorn

server = None

def run_server():
    global server
    config = uvicorn.Config(app, host="127.0.0.1", port=8181, log_level="warning")
    server = uvicorn.Server(config)
    asyncio.run(server.serve())

thread = threading.Thread(target=run_server, daemon=True)
thread.start()
time.sleep(2)  # Wait for the server to start

# Now test with the MCP client
from mcp import ClientSession
try:
    from mcp.client.streamable_http import streamable_http_client
except ImportError:
    from mcp.client.streamable_http import streamablehttp_client as streamable_http_client
import nest_asyncio
nest_asyncio.apply()

async def test_local_server():
    async with streamable_http_client("http://127.0.0.1:8181/product-catalog/mcp") as (read, write, _):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # List tools
            response = await session.list_tools()
            print(f"✅ MCP server running locally with {len(response.tools)} tools:")
            for tool in response.tools:
                print(f"   🔧 {tool.name}")

            # Test: get_categories
            result = await session.call_tool("get_categories", {})
            print(f"\n📦 Categories: {result.content[0].text}")

            # Test: search_products
            result = await session.call_tool("search_products", {"query": "wireless"})
            print(f"\n🔍 Search 'wireless': {result.content[0].text}")

            # Test: check_stock  
            result = await session.call_tool("check_stock", {"product_id": "PROD-001"})
            print(f"\n📊 Stock PROD-001: {result.content[0].text}")

asyncio.run(test_local_server())

# Shutdown the server
if server:
    server.should_exit = True
print("\n✅ Local MCP server test completed successfully!")

<a id='2'></a>
### 2️⃣ Deploy to Azure using 🦾 Bicep

This step provisions the complete infrastructure using [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) — Azure API Management (v2 with native MCP support), Container App environment, AI Foundry, monitoring, and networking.

APIM is configured with a **native MCP-type API** that speaks Streamable HTTP to the backend, plus an **Inference API** for LLM access — both governed by the same policies, subscriptions, and monitoring.

> 💡 Modify [main.bicep](main.bicep) to customize the deployment (SKU, policies, additional APIs).

In [ ]:
# Create the resource group if it doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "aiServicesConfig": { "value": aiservices_config },
        "modelsConfig": { "value": models_config },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "inferenceAPIPath": { "value": inference_api_path },
        "inferenceAPIType": { "value": inference_api_type },
        "foundryProjectName": { "value": foundry_project_name },
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the APIM gateway URL, subscription keys, and resource names from the Bicep deployment. These are used in all subsequent steps to connect to your modernized API through APIM.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}",
    f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    ai_foundry_project_endpoint = utils.get_deployment_output(output, 'foundryProjectEndpoint', 'AI Foundry Project Endpoint')
    log_analytics_id = utils.get_deployment_output(output, 'logAnalyticsWorkspaceId', 'Log Analytics Id')
    apim_resource_name = utils.get_deployment_output(output, 'apimResourceName', 'APIM Resource Name')
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    for subscription in apim_subscriptions:
        subscription_name = subscription['name']
        subscription_key = subscription['key']
        utils.print_info(f"Subscription Name: {subscription_name}")
        utils.print_info(f"Subscription Key: ****{subscription_key[-4:]}")
    api_key = apim_subscriptions[0].get("key")  # default api key to the first subscription key
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')
    container_registry_name = utils.get_deployment_output(output, 'containerRegistryName', 'Container Registry Name')
    product_catalog_containerapp_name = utils.get_deployment_output(output, 'productCatalogContainerAppResourceName', 'Product Catalog Container App Name')

<a id='4'></a>
### 4️⃣ Build and deploy the Product Catalog MCP Server

Build the container image in Azure Container Registry and deploy it to the Container App. Once running, APIM will route MCP traffic to this backend — your REST API is now an MCP server, governed by APIM.

In [ ]:
build = build + 1  # increment the build number

utils.run(f"az acr build --image {product_catalog_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {product_catalog_mcp_server_src}/Dockerfile {product_catalog_mcp_server_src}/. --no-logs",
          "Product Catalog MCP Server image was successfully built", "Failed to build the Product Catalog MCP Server image")

utils.run(f'az containerapp update -n {product_catalog_containerapp_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{product_catalog_mcp_server_image}:v0.{build}"',
          "Product Catalog MCP Server deployment succeeded", "Product Catalog MCP Server deployment failed")

<a id='testconnection'></a>
### 🧪 Discover MCP tools through Azure API Management

Connect to the APIM gateway and list all available MCP tools. This validates that APIM is correctly routing Streamable HTTP traffic to the backend MCP server and that tool metadata is discoverable.

The `api-key` header carries the APIM subscription key — every request is **authenticated and governed** by APIM policies before reaching the backend.

> 💡 The same MCP endpoint can be used in VS Code, the MCP Inspector, or any MCP-compatible client: `{apim_gateway_url}/product-catalog/mcp`

In [ ]:
import os, json, asyncio, time, requests, httpx
from mcp import ClientSession
from mcp.client.streamable_http import streamable_http_client
import nest_asyncio
nest_asyncio.apply()

# APIM native MCP type APIs serve the MCP protocol at the API path directly (no /mcp suffix)
mcp_url = f"{apim_resource_gateway_url}/product-catalog"
print(f"🔍 Connecting to APIM MCP endpoint: {mcp_url}")

async def list_tools(server_url, authorization_header=None):
    headers = {
        "api-key": api_key,
        "Accept": "application/json, text/event-stream",
    }
    if authorization_header:
        headers["Authorization"] = authorization_header
    tools = []
    try:
        http_client = httpx.AsyncClient(headers=headers, timeout=30)
        async with streamable_http_client(server_url, http_client=http_client) as (read_stream, write_stream, _):
            async with ClientSession(read_stream, write_stream) as session:
                await session.initialize()
                response = await session.list_tools()
                tools = response.tools
    except Exception as e:
        print(f"❌ Error: {e}")
    if tools:
        print(f"✅ Connected to MCP server via APIM")
        print("⚙️ Tools:")
        for tool in tools:
            print(f"  - {tool.name}")
            print(f"     Input Schema: {tool.inputSchema}")
    return tools

tools = asyncio.run(list_tools(mcp_url))
print(f"✅ Connection closed")

<a id='mcpcall'></a>
### 🧪 Call MCP tools end-to-end through Azure API Management

This cell opens an MCP session to the APIM gateway and **invokes all 5 tools** — listing categories, browsing products, searching by keyword, and checking stock. Every call flows through APIM's policy engine, demonstrating that your modernized API works seamlessly for AI agent consumption.

The `api-key` header authenticates each request via the APIM subscription — the same security model you'd use for any API behind APIM.

In [ ]:
import asyncio, json, httpx
from mcp import ClientSession
from mcp.client.streamable_http import streamable_http_client
import nest_asyncio
nest_asyncio.apply()

async def call_mcp_tools(server_url):
    http_client = httpx.AsyncClient(headers={"api-key": api_key}, timeout=30)
    async with streamable_http_client(server_url, http_client=http_client) as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()

            # 1️⃣ get_categories – list all product categories
            result = await session.call_tool("get_categories", {})
            categories = result.content[0].text
            print(f"📦 Categories: {categories}\n")

            # 2️⃣ get_products_by_category – fetch all Electronics products
            result = await session.call_tool("get_products_by_category", {"category": "Electronics"})
            products = json.loads(result.content[0].text.replace("'", '"'))
            print(f"🖥️  Electronics products ({len(products)} items):")
            for p in products:
                print(f"   • {p['name']} — ${p['price']} (stock: {p['stock']})")
            print()

            # 3️⃣ get_product – get details for a single product
            result = await session.call_tool("get_product", {"product_id": "PROD-003"})
            print(f"🔎 Product detail (PROD-003): {result.content[0].text}\n")

            # 4️⃣ search_products – keyword search
            result = await session.call_tool("search_products", {"query": "wireless"})
            print(f"🔍 Search 'wireless': {result.content[0].text}\n")

            # 5️⃣ check_stock – stock availability
            result = await session.call_tool("check_stock", {"product_id": "PROD-001"})
            print(f"📊 Stock (PROD-001): {result.content[0].text}")

print(f"🔗 MCP endpoint: {mcp_url}\n")
asyncio.run(call_mcp_tools(mcp_url))
print("\n✅ All 5 MCP tool calls through APIM succeeded!")

<a id='inspector'></a>
### 🧪 (Optional) Interactive testing with the [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector)

The MCP Inspector provides a browser-based UI for exploring and testing MCP servers interactively — useful for debugging tool schemas, testing edge cases, and verifying your modernized API behaves correctly.

#### Steps:
1. Run `npx @modelcontextprotocol/inspector` in a terminal
2. Open the provided URL in a browser
3. Set the transport type to **Streamable HTTP**
4. Enter the MCP server URL and click **Connect**
5. Navigate to the **Tools** tab to discover and invoke available tools

<a id='agentframework'></a>
### 🧪 AI Agent using MCP Tools via Azure API Management

This is where the modernization comes full circle — an AI agent powered by the [Microsoft Agent Framework](https://github.com/microsoft/agent-framework) discovers and invokes your Product Catalog tools through APIM, with **both LLM inference and MCP tool calls** governed by the same API Management instance.

The [Microsoft Agent Framework](https://learn.microsoft.com/en-us/agent-framework/) natively supports MCP through `MCPStreamableHTTPTool`, connecting agents to MCP servers over Streamable HTTP transport. The agent autonomously decides which tools to call based on the user's natural language query.

> 💡 This demonstrates the full **API modernization value**: your original REST API, unchanged in its business logic, is now accessible to AI agents — with enterprise security, monitoring, and governance at every layer.

In [ ]:
# type: ignore
import asyncio
import httpx
from agent_framework import ChatAgent, MCPStreamableHTTPTool
from agent_framework.azure import AzureOpenAIChatClient
import nest_asyncio
nest_asyncio.apply()

async def main():
    # Create an agent with MCP tools - both LLM and MCP are routed through Azure API Management
    async with ChatAgent(
        chat_client=AzureOpenAIChatClient(
            endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
            deployment_name=models_config[0]['name'],
            api_key=api_key,
            api_version=inference_api_version,
        ),
        name="ProductAgent",
        instructions="You are a helpful product catalog assistant. Use the available tools to search and retrieve product information.",
        tools=MCPStreamableHTTPTool(
            name="ProductCatalog",
            url=f"{apim_resource_gateway_url}/product-catalog",
            description="Product Catalog tools for searching and retrieving product information",
            http_client=httpx.AsyncClient(headers={"api-key": api_key}),
            approval_mode="never_require",
        ),
    ) as agent:
        # Query 1: Search for electronics products and check stock
        query1 = "What electronics products do you have and which ones are in stock?"
        print(f"# User: {query1}")
        result1 = await agent.run(query1)
        print(f"# {agent.name}: {result1.text}\n")

        # Query 2: Search by keyword
        query2 = "Search for products related to 'charging' and tell me about them."
        print(f"# User: {query2}")
        result2 = await agent.run(query2)
        print(f"# {agent.name}: {result2.text}\n")

asyncio.run(main())

<a id='vscode'></a>
### 🧪 Configure the MCP Server in VS Code for GitHub Copilot Agent Mode

Integrate your modernized API directly into your development workflow. Once configured, GitHub Copilot can discover and call your Product Catalog tools through APIM — bringing AI-powered API interaction right into your editor.

1. [Configure the MCP Server in VS Code](https://code.visualstudio.com/docs/copilot/chat/mcp-servers#_add-an-mcp-server)
2. Add the following to your `.vscode/mcp.json`:
```json
{
    "servers": {
        "product-catalog": {
            "type": "http",
            "url": "<apim_resource_gateway_url>/product-catalog/mcp",
            "headers": {
                "api-key": "<your-apim-subscription-key>"
            }
        }
    }
}
```
3. In Copilot chat, try: *"Search for products related to USB and check their stock"*
4. Copilot will discover and call the `search_products` and `check_stock` tools via APIM

In [ ]:
print(f"📋 MCP Server URL for VS Code: {apim_resource_gateway_url}/product-catalog/mcp")
print(f"📋 APIM Gateway URL: {apim_resource_gateway_url}")

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, delete all deployed resources to avoid extra charges. Each run creates an isolated resource group with a unique suffix, so cleanup only affects resources from this specific run.

In [ ]:
# ⚠️ This will delete ALL resources created during this notebook run
output = utils.run(f"az group delete --name {resource_group_name} --yes --no-wait",
    f"Resource group '{resource_group_name}' deletion started", f"Failed to delete resource group '{resource_group_name}'")